# Install Libraries

In [ ]:
# !pip install huggingface_hub > /dev/null
# !pip install git+https://github.com/huggingface/transformers.git > /dev/null
# !pip install accelerate -U > /dev/null
!pip install datasets==2.8.0 > /dev/null
!pip install jiwer > /dev/null
# !pip uninstall wandb -y

In [ ]:
# # Detect hardware, return appropriate distribution strategy
# import tensorflow as tf
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hugging_face_key")
login(secret_value_0)

In [ ]:
%%capture
!apt install git-lfs

In [ ]:
# !pip uninstall transformers -y
# !pip install transformers==4.20.0
# !pip install https://github.com/kpu/kenlm/archive/master.zip
# !pip install pyctcdecode

# from transformers.file_utils import cached_path, hf_bucket_url
# from importlib.machinery import SourceFileLoader
# from transformers import Wav2Vec2ProcessorWithLM

In [ ]:
from IPython.lib.display import Audio
import torchaudio
import torch
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict
import json
import pandas as pd
import os
os.environ["WANDB_DISABLED"] = "true"
from datasets import load_metric
wer_metric = load_metric("wer")

# Load datasets

In [ ]:
# data = load_dataset("foxxy-hm/slu-augmented-data")
train = load_from_disk("/kaggle/input/soict2023-slu-preprocess-and-augmentation-stage-1/data/train.dataset", keep_in_memory=True)
valid = load_from_disk("/kaggle/input/soict2023-slu-preprocess-and-augmentation-stage-1/data/valid.dataset", keep_in_memory=True)
data = DatasetDict({
    "train": train,
    "valid": valid
})
train.cleanup_cache_files()
valid.cleanup_cache_files()

In [ ]:
data

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf
import torch

cache = "./cache"
processor = Wav2Vec2Processor.from_pretrained("foxxy-hm/wav2vec2-base-finetune-vi-v6")
# model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h",
#                                        gradient_checkpointing=True,
#                                         ctc_loss_reduction="mean",
#                                         pad_token_id=processor.tokenizer.pad_token_id)
# model_name = "nguyenvulebinh/wav2vec2-large-vi"
model_name = "foxxy-hm/wav2vec2-base-finetune-vi-v6"
model = Wav2Vec2ForCTC.from_pretrained(model_name, pad_token_id=processor.tokenizer.pad_token_id, 
#                                       gradient_checkpointing=True,
                                       ctc_loss_reduction="mean",
                                       vocab_size=len(processor.tokenizer.get_vocab())-2, # 110
                                      ignore_mismatched_sizes=True)

In [ ]:
# processor.push_to_hub("wav2vec2-base-finetune-vi-v5")

Kiểm tra rằng toàn bộ các samples có sampling rate là 16kHz. Sau đó, extract `input_values` from loaded audio file. Cuối cùng, encode the transcriptions thành label ids.

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0], padding="longest").input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
data_prepared = data.map(prepare_dataset, remove_columns=data.column_names["train"], batch_size=8, batched=True)
data.cleanup_cache_files()

## Zero-shot inference

In [ ]:
# def map_to_result(batch):
#   model.to("cuda")
#   input_values = processor(
#       batch["speech"],
#       sampling_rate=batch["sampling_rate"],
#       return_tensors="pt",
#         padding="longest"
#   ).input_values.to("cuda")

#   with torch.no_grad():
#     logits = model(input_values).logits

#   pred_ids = torch.argmax(logits, dim=-1)
#   batch["pred_str"] = processor.batch_decode(pred_ids)[0]

#   return batch

In [ ]:
# results = data["test"].map(map_to_result)

In [ ]:
# show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

In [ ]:
# print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

```
Test WER: 0.407
```

# Training Model

## Setup Trainer

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
model.freeze_feature_extractor()

In [ ]:
%cd /kaggle/working/

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-base-finetune-vi-v6",
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=20,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=6e-5,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  gradient_checkpointing=True,
  push_to_hub=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared["train"],
    eval_dataset=data_prepared["valid"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/kaggle/working")
# trainer.push_to_hub("wav2vec2-base-finetune-vi-v5")

# Inference

In [ ]:
# !pip install jiwer
# from datasets import load_metric
# wer_metric = load_metric("wer")

In [ ]:
# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2Config
# import torch


# processor = Wav2Vec2Processor.from_pretrained("/kaggle/working/", local_files_only=True)
# model = Wav2Vec2ForCTC.from_pretrained("/kaggle/working/", local_files_only=True)

In [ ]:
# import random

# model.to("cuda")
# n = random.randint(0, len(data["valid"]))
# batch = data["valid"][n]
# input_values = processor(
#   batch["speech"],
#   sampling_rate=batch["sampling_rate"],
#   return_tensors="pt"
# ).input_values.to("cuda")

# with torch.no_grad():
#     logits = model(input_values).logits

# pred_ids = torch.argmax(logits, dim=-1)
# pred = processor.batch_decode(pred_ids)[0]
# print(n, pred)